'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

![Bottleneck_features_original](./pictures/vgg16_modified.png)

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense

Using TensorFlow backend.


In [8]:
# path to the model weights files.
weights_path = 'vgg16_weights.h5' 
# 'vgg16_weights.h5'  download link: https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
top_model_weights_path = 'bottleneck_fc_model.h5'
# 'bottleneck_fc_model.h5' from the result of classifier_from_little_data_script_2
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [3]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

Model loaded.


In [4]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [5]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [6]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

model.summary()

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
___________________________________________________________

In [9]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=2)

Epoch 1/50
11s - loss: 0.1767 - acc: 0.9340 - val_loss: 0.2857 - val_acc: 0.9262
Epoch 2/50
10s - loss: 0.1650 - acc: 0.9370 - val_loss: 0.3118 - val_acc: 0.9084
Epoch 3/50
10s - loss: 0.1443 - acc: 0.9525 - val_loss: 0.3161 - val_acc: 0.9173
Epoch 4/50
10s - loss: 0.1331 - acc: 0.9555 - val_loss: 0.2941 - val_acc: 0.9288
Epoch 5/50
10s - loss: 0.0879 - acc: 0.9675 - val_loss: 0.3051 - val_acc: 0.9275
Epoch 6/50
10s - loss: 0.0906 - acc: 0.9655 - val_loss: 0.5398 - val_acc: 0.8779
Epoch 7/50
10s - loss: 0.0942 - acc: 0.9625 - val_loss: 0.3901 - val_acc: 0.9148
Epoch 8/50
10s - loss: 0.0743 - acc: 0.9715 - val_loss: 0.3263 - val_acc: 0.9071
Epoch 9/50
10s - loss: 0.0920 - acc: 0.9705 - val_loss: 0.3483 - val_acc: 0.9160
Epoch 10/50
10s - loss: 0.0648 - acc: 0.9785 - val_loss: 0.3489 - val_acc: 0.9173
Epoch 11/50
10s - loss: 0.0489 - acc: 0.9790 - val_loss: 0.4499 - val_acc: 0.9211
Epoch 12/50
10s - loss: 0.0612 - acc: 0.9775 - val_loss: 0.3002 - val_acc: 0.9313
Epoch 13/50
10s - loss: 0